# IP Day 0 Activity
## Advanced Algorithms Spring 2023

In [ ]:
!pip install mip

# Example 0: The 0/1 Knapsack Problem

Let's think about the following formulation of the 0/1 Knapsack Problem (following the in-class worksheet!):

#### Maximize:
$$
8x_1+12x_2+7x_3+15x_4+12x_5
$$

#### Such that:
$$
4x_1+8x_2+3x_3+6x_4+5x_5\leq15 \\
x_j\in\{0,1\}\ \forall j
$$

Writing the formulation out is nice enough, but let's solve this Integer Programming problem. We can do so using an existing library specifically geared for solving problems like these. To start, we'll need to rewrite the problem a bit to suit the library - so let's do that. First, we write the IP form of the general 0/1 knapsack problem:

$$
\max \sum_{i\in I} p_i\cdot x_i \\
s.t. \sum_{i\in I}w_i \cdot x_i  \leq c \\
x_i \in \{0,1\} \forall i \in I
$$

where:
- $c$ = the capacity   
- $p_i$ = the value
- $x_i$ = the item

Why did we do this? It's because it's a nice general representation for this type of problem. You don't have to do it like this, but it sure makes life easy.

Next, let's match up our specific problem with this form:

|  1  |  2  |  3  |  4  |  5  |
|-----|-----|-----|-----|-----|
| 8   | 12  | 7   | 15  | 12  |
| 4   | 8   | 3   | 6   | 5   |

We'll write that into a vector-form:
$$
p = [8,12,7,15,12] \\
w = [4, 8, 3, 6, 5] \\
c = 15
$$

Step 0: Define the problem!

We can now begin the main IP solving portion. We'll use the `python-mip` library, a highly polished and efficient implementation of the Branch-and-Cut solver for IP problems. It's a very nice example of how you don't always necessarily need to implement the algorithms you learn, since people have done that work for you. You will learn how branch and cut works next class, but for now, just focus on using the library to solve problems.

Step 0 (part 2): Import the library.

In [2]:
from mip import Model, xsum, maximize, BINARY

We've imported a bunch of stuff from the library, which you'll see us use shortly.

Step 1: Create a "model" to hold our problem definition.

Step 2: Add as many variables (all the $x_i$) as you need to the problem definition (the model).

Step 3: Define the model's objective function.

Step 4: Add the constraint function(s) to the model.

Step 5: Run the optimizer!

Step 6 (FINAL): Display the results!

**Seems like we can add items 0, 3, and 4, to the knapsack!**

# Example 2: Investing $$

Let's think about the following formulation of the investment problem (following the in-class worksheet!):

#### Maximize:

$8x_1 + 11x_2 + 6x_3 + 4x_4$

#### Subject To 
$5x_1 + 7x_2 + 4x_3 + 3x_4 \leq 14$    
$x_j \in \{0, 1\}$ with $j = \{1, 2, 3, 4\}$   

### Let's put it in the general form!
$$
\max \sum_{j=1}^{4} v_i \cdot x_i \\
s.t. \\
\sum_{j=1}{4} i_j \cdot x_i \leq m \\
x_j \in \{0, 1\} \forall j
$$

where:
- $v$ = the value of the investment
- $i_j$ = required amount for that investment
- $x_j$ the investment opportunity 
- $m$ = the amount of money we have to invest

In [ ]:
# View the results
selected = [i for i in I if x[i].x >= 0.99]
print(selected)
print(m.objective_value)

# Example 3: Frequency Assignment

Let's examine a common problem among many wireless routing companies - frequency assignment. It's the problem of deciding which local area will use which frequency bands for their communications, while ensuring there's no overlap or confusion between neighboring regions. A good example of this would be your local radio service.

For our particular in-class example, we're going to use an examination of Philadelphia's cell phone network done in 1973 (source paper is cited at the bottom of all this):
 - For efficient coverage of area, let's describe each region in the mobile network as a hexagon.
 - Each hexagon will consist of two numbers: its index number (label), as well as the number of communication channels the region requires.
We'll represent this setup as seen below:

![philly](philly-frequency.png)

Each region (cell) has at most 6 neighboring regions; cell #2 is the region with the largest demand, while cell #5 has the most neighboring cells.

Here's some additional information about the structure and selection of frequencies in each cell:
 - We can assign to each cell frequencies from the list $$U=\{1,2,\dots,\bar{u}\}$$ where $\bar{u}$ represents the largest frequency you can assign. We use nice numbers for our frequencies (1, 2, etc.) for ease of notation, but without loss of information.
 - For each cell, the frequencies have to be at least 3 apart. For example, a cell can be assigned frequencies 1 and 4, but not frequencies 1 and 3.
 - The frequencies of neighboring cells have to be at least 2 apart. For example, cell #1 can't have frequency 7 if cell #2 has frequency 8.

**Your exercise**: Devise the general mathematical formulation for a problem like this.

IP Formulation:

$$\text{Minimize} \quad z$$

$$ \text{Such that} \\
\sum_{c=1}^{\bar{u}}x_{(i,c)} = r_i \forall i \in N \\
z \geq c\cdot x_{(i,c)} \forall i \in N, c \in U \\
x_{(i,c)} + x_{(j,c')} \leq 1 \forall (i,j,c,c') \in N \times N \times U \times U : i \neq \land |c-c'| < d_{(i,j)} \\
x_{(i,c)} + x_{(i,c')} \leq 1 \forall i,c \in N \times U, c' \in \{c, c+1+\dots, \min(c+d_{i,i},\bar{u})\} \\
x_{(i,c)} \in \{0,1\} \forall i \in N, c \in U \\
z \geq 0
$$

Now, let's get to implementing a solution for this. We'll use the same `python-mip` library as before.

Step 0 - import the necessary things. We've done that for you (maybe these are clues? Who knows).

In [24]:
from itertools import product
from mip import Model, xsum, minimize, BINARY

Step 0 (part 2): Let's define the problem. Here's what we'd suggest:
 * Something that contains the number of required channels per region
 * Something that defines the necessary channel distance between any two regions
 * The number of regions

We've defined the set $U$ for you (therefore also including $\bar{u}$).

In [26]:
# in complete applications this upper bound should be obtained from a feasible
# solution produced with some heuristic
U = range(sum(d[i][j] for (i, j) in product(N, N)) + sum(el for el in r))

Step 1: Create a model to hold our problem definition.

Step 2: Add as many variables (all the $x_{(i,c)}$, etc.) as you need to the problem definition (the model).

Step 3: Define the model's objective function.

Step 4: Add the constraint function(s) to the model.

Step 5: Run the optimizer!

Step 6 (FINAL): Display the results! Doing this is unnecessarily complicated, so we wrote it out for you.

In [32]:
if m.num_solutions:
    for i in N:
        print('Channels of node %d: %s' % (i, [c for c in U if x[i][c].x >=
                                               0.99]))

Channels of node 0: [0, 6, 12]
Channels of node 1: [4, 10, 16, 22, 38]
Channels of node 2: [2, 8, 14, 20, 26, 31, 36, 40]
Channels of node 3: [1, 6, 13]
Channels of node 4: [4, 15, 18, 22, 29, 34]
Channels of node 5: [2, 8, 20, 26, 31]
Channels of node 6: [0, 6, 12, 18, 24, 29, 34]
Channels of node 7: [8, 11, 18]


# Appendix A
### Source list:
#### Knapsack Problem
Original source code (adapted to our example): https://docs.python-mip.com/en/latest/examples.html
#### Frequency Assignment 
Original paper: L.G. Anderson: A Simulation Study of some Dynamic Channel Assignment Algorithms in a High Capacity Mobile Telecommunications System. IEEE Transactions on Communications, 21:1294–1301. 1973.

Source code: https://docs.python-mip.com/en/latest/examples.html

# Appendix B: Traveling Salesman Problem (do this on your own)
Source for example and code: https://docs.python-mip.com/en/latest/examples.html

![tsp](tsp.png)

We're going to use an example that's on the `python-mip` website, because it's done nicely. The source is linked right above - look through that for the problem formulation and information.

Import the necessary things. We've done that for you

In [9]:
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY

Define the problem

In [2]:
# names of places to visit

# distances in an upper triangular matrix

# number of nodes and list of vertices

# distances matrix

Define the model and constraints

Run the model and find a solution